<a href="https://colab.research.google.com/github/PatrickRuan/3rd-ML100Days/blob/master/homework/Day_029_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [17]:
from google.colab import drive
drive.mount('/content/drive/')

root = 'drive/My Drive/'
learning = root + '學習/'

# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = learning + 'Part02/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here 
"""
df['Pclass']=df.Pclass.fillna('None')
mean_df = df.groupby(['Pclass'])['Age'].mean().reset_index()
mode_df = df.groupby(['Pclass'])['Age'].apply(lambda x:x.mode()[0]).reset_index()
median_df = df.groupby(['Pclass'])['Age'].median().reset_index()
max_df = df.groupby(['Pclass'])['Age'].max().reset_index()

temp = pd.merge(mean_df, mode_df, how='left', on=['Pclass'])
temp = pd.merge(temp, median_df, how='left', on=['Pclass'])
temp = pd.merge(temp, max_df, how='left', on=['Pclass'])
temp.columns = ['Pclass', 'Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max']
temp

,Pclass,Age_Mean,Age_Mode,Age_Median,Age_Max
0,1,38.233441,35.0,37.0,80.0
1,2,29.877630,24.0,29.0,70.0
2,3,25.140620,22.0,24.0,74.0


In [19]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()

df = pd.merge(df, temp, how='left', on=['Pclass'])
df = df.drop(['Pclass'] , axis=1)
df_minus = df.drop(['Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max'] , axis=1)
print(df.head())
print(df_minus.head())

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

    Age  SibSp  Parch     Fare   Age_Mean  Age_Mode  Age_Median  Age_Max
0  22.0      1      0   7.2500  25.140620      22.0        24.0     74.0
1  38.0      1      0  71.2833  38.233441      35.0        37.0     80.0
2  26.0      0      0   7.9250  25.140620      22.0        24.0     74.0
3  35.0      1      0  53.1000  38.233441      35.0        37.0     80.0
4  35.0      0      0   8.0500  25.140620      22.0        24.0     74.0
    Age  SibSp  Parch     Fare
0  22.0      1      0   7.2500
1  38.0      1      0  71.2833
2  26.0      0      0   7.9250
3  35.0      1      0  53.1000
4  35.0      0      0   8.0500


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [20]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df_minus)
model = LogisticRegression()
cross_val_score(model, train_X, train_Y, cv=5).mean()


0.6408846483794697

In [21]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
model = LogisticRegression()
cross_val_score(model, train_X, train_Y, cv=5).mean()


0.6959981729116419

In [22]:
# 新特徵 + 邏輯斯迴歸  如果把 Age 也去除, 非樹形模型，我們嘗試少一點共線性，這樣思考對吧?
"""
Your Code Here
"""
df = df.drop(['Age'], axis=1)
train_X = MMEncoder.fit_transform(df)
model = LogisticRegression()
cross_val_score(model, train_X, train_Y, cv=5).mean()

0.6937511946824353